In [7]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
# from mplfinance import candlestick_ohlc
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates



import numpy as np

from os.path import exists

from datetime import datetime
import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import seaborn as sns

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 300})


In [56]:
df = pd.read_json(r'./freq-user-data/data/kucoin/ALGO_USDT-1h.json')
df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

df['date'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
df.set_index(pd.DatetimeIndex(df["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')


In [57]:
# pd.Timestamp('now').floor('D') + pd.Timedelta(-7, unit='D')

# start_time = datetime(2021,5,1)

start_time = pd.Timestamp('now') + pd.Timedelta(-80, unit='W')
# start_time
ddf = df.loc[df.index >= start_time].copy()
# ddf = df.loc[df['time'] >= start_time]
# ddf
dlen = len(ddf.index)
dlen

9357

In [66]:
wdf.index.values

array(['2021-01-01T00:00:00.000000000', '2021-01-01T01:00:00.000000000',
       '2021-01-01T02:00:00.000000000', ...,
       '2022-01-25T18:00:00.000000000', '2022-01-25T19:00:00.000000000',
       '2022-01-25T20:00:00.000000000'], dtype='datetime64[ns]')

In [96]:
strategy = '2-emas'
strategy_params_json = f'./strategy-params/{strategy}.json'

fee:float = 0.001
# capital = 10000
# risk_per_trade_perc = 0.01
# risk = 0.01
dlen = len(ddf)

sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')

strategy_params = {
    'mav1': {'type': 'int', 'value': 21, 'step': 1},
    'mav2': {'type': 'int', 'value': 50, 'step': 1},
    'mav3': {'type': 'int', 'value': 50, 'step': 5},
    'mav1mav2': {'type': 'float', 'value': 0.01, 'step': 0.0005},
    'mav2mav3': {'type': 'float', 'value': 0.01, 'step': 0.0005},
    'stop_gap': {'type': 'float', 'value': 0.001, 'step': 0.0005},
    'slipage': {'type': 'float', 'value': 0.001, 'step': 0.0005},
    'risk': {'type': 'float', 'value': 0.01, 'step': 0.005},
    'r2r': {'type': 'float', 'value': 2./1., 'step': 0.05},
}
for pk, pv in strategy_params.items():
    pv['wdg'] = globals()[(pv['wdg'] if 'wdg' in pv else pv['type'].capitalize() + 'Text')](
        description=pk,disabled=False,continuous_update=False,
        value=pv['value'], step=pv['step']
    )
strategy_params = {"w2log": {'wdg': sl_n2}, "w": {'wdg': sl_w}, **strategy_params}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

wst = wed = wsz = buy_sig = g_buy_sig_cum = buy_cond = False
enter_sig_ind = enter_sig_arr = exit_sig_arr = False
wdf = wadf = False
tdf = pd.DataFrame(columns = ['side', 'enter_price', 'buy_price', 'exit_price', 'sell_price', 'qty', 'pnl', 'capital'])
def printer(w2log, w, mav1, mav2, mav3, mav1mav2, mav2mav3, r2r, stop_gap, slipage, risk):
    # global capital
    capital = 10000
    global wst, wed, wsz, wdf, tdf
    global enter_sig_ind, enter_sig_arr, exit_sig_arr, stop_sig_arr, target_sig_arr
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))

    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = ddf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)

    # wdf['dnwick'] = wdf.close - wdf.low
    wdf['mav1'] = wdf.ta.ema(length=mav1, sma=False)
    wdf['mav2'] = wdf.ta.ema(length=mav2, sma=False)
    wdf['mav3'] = wdf.ta.ema(length=mav3, sma=False)
    # wdf['c2mav2'] = abs(wdf.close - wdf.mav2) / wdf.mav2
    wdf['mav1mav2'] = (wdf.mav1 - wdf.mav2) / wdf.mav2
    wdf['mav2mav3'] = (wdf.mav2 - wdf.mav3) / wdf.mav3

    wdf['direction'] = np.where(wdf.close > wdf.open, 1, -1)

    buy_sig = (
        # (wdf.low2body >= lw2b) & 
        # (wdf.direction == -1) &
        (wdf.low < wdf.mav2) &
        (wdf.close >= wdf.mav2) &
        (wdf.open >= wdf.close) &
        # (wdf.mav1 > wdf.mav2) &
        # (wdf.mav2 > wdf.mav3) &
        (wdf.mav1mav2 >= mav1mav2) &
        (wdf.mav2mav3 >= mav2mav3) 
        
    ).shift(1, fill_value=False)


    # sell_sig = (
    #     (wdf.high2body >= hw2b) & 
    #     (wdf.direction == 1)
        
    # ).shift(1, fill_value=False)

    # buy_sig = buy_cond & (~buy_cond).shift(1, fill_value=False)

    enter_sig_arr = np.full(wlen, False)
    exit_sig_arr = np.full(wlen, False)
    in_pos_arr = np.full(wlen, 0)
    roi_ret_arr = np.full(wlen, 0.)
    pnl_arr = np.full(wlen, 0.)
    cap_arr = np.full(wlen, 0.)

    enter_price_arr = np.full(wlen, np.nan)
    stop_price_arr = np.full(wlen, np.nan)
    target_price_arr = np.full(wlen, np.nan)
    exit_price_arr = np.full(wlen, np.nan)
    capital_arr = np.full(wlen, np.nan)

    # pos_arr = np.full(wlen, 0.)
    vopen, vclose, vhigh, vlow = wdf.open, wdf.close, wdf.high, wdf.low
    vdates = wdf.index.values
    mav1a, mav2a = wdf.mav1, wdf.mav2

    is_long = is_short = in_pos = enter_date = False
    enter_price, stop_price, target_price, buy_price, sell_price, pos_qty = 0., 0., 0., 0., 0., 0.
    for i in range(max(mav1,mav2), wlen):
        in_pos_arr[i] = in_pos

        if not in_pos and buy_sig[i]:
            is_long, in_pos = True, True
            enter_date = vdates[i]
            enter_sig_arr[i] = True
            enter_price = vopen[i] 
            buy_price = enter_price * (1 + fee) * (1 + slipage)
            stop_price = vlow[i-4:i-1].min() * (1. - stop_gap)
            target_price = enter_price + (enter_price - stop_price) * r2r
            pos_qty = capital * risk /(enter_price - stop_price)
            pos_qty = max(pos_qty, capital / 2 / enter_price)
            enter_price_arr[i] = enter_price
            target_price_arr[i] = target_price
            stop_price_arr[i] = stop_price

        if is_long:
            exit_price, pnl = 0., 0.
            if vlow[i] <= stop_price:
                exit_price = stop_price
            elif vhigh[i] >= target_price:            
                exit_price = target_price 
            # elif vopen[i-1] < enter_price:
            #     exit_price = vopen[i]
            elif mav1a[i-1] < mav2a[i-1] and enter_price > vopen[i]:
                exit_price = vopen[i]
            # elif sell_sig[i]:
            #     exit_price = vopen[i]
            # elif green_ins_green[i-1]:
            #     exit_price = vopen[i]
            
            if exit_price != 0:
                sell_price = exit_price * (1 - fee) * (1 - slipage)
                exit_price_arr[i] = exit_price
                exit_sig_arr[i] = True
                pnl = (sell_price - buy_price) * pos_qty 
                pnl_arr[i] = pnl
                exit_date = vdates[i]
                capital += pnl
                tdf = tdf.append({'side': 'long', 'enter_date': enter_date, 'enter_price': enter_price, 'buy_price': buy_price, 
                    'exit_date': exit_date, 'exit_price': exit_price, 'sell_price': sell_price, 'qty': pos_qty, 'pnl': pnl, 'capital':capital}, ignore_index=True)
                # roi_ret_arr[i] = ret / enter_price - 2 * fee
                in_pos, is_long = False, False

    wdf['pnl'] = pnl_arr
    wdf['cum_pnl'] = wdf.pnl.cumsum()
    # wdf['cap'] = capital
    # wdf['in_pos'] = in_pos_arr
    wdf['enter_sig'] = enter_sig_arr
    wdf['exit_sig'] = exit_sig_arr
    wdf['enter_price'] = np.where(enter_sig_arr, enter_price_arr, np.nan)
    wdf['target_price'] = np.where(enter_sig_arr, target_price_arr, np.nan)
    wdf['stop_price'] = np.where(enter_sig_arr, stop_price_arr, np.nan)
    wdf['exit_price'] = np.where(exit_sig_arr, exit_price_arr, np.nan)

    enter_sig_ind = wdf[wdf.enter_sig].index
    exit_sig_ind = wdf[wdf.exit_sig].index
    
    # positions_start_row = enter_groups.head(1)
    # positions_end_row = exit_groups.head(1)
    # positions_min_poscumlret = exit_groups.poscumlret.min()
    # positions_max_poscumlret = exit_groups.poscumlret.max()

    # wdf['poscumplret'] = wdf.totcumlret * wdf.in_pos.astype(int)

    # wexits = wdf.loc[exit_sig_ind]
    # wprofits = wexits[wexits.pnl > 0]
    # wlosses = wexits[wexits.pnl <= 0]

    
#     with open(f'./strategy-results/{datetime.now():strat_%Y-%m-%d}.log', "a") as f: 
#         f.write(json.dumps(dict({   
#             "dt": f'{datetime.now():%Y-%m-%d %HH:%MM:%SS}',
#             "period_start": f'{ddf.index[0]}',
#             "period_end": f'{ddf.index[-1]}',
#             "params": dict({ 'sl': sl, 'ts': ts, 'tp': tp, 'ma1n': ma1n, 'ma2n': ma2n, 'ma1c': ma1c, 'ma2c': ma2c, 'trn': trn, 'mrn': mrn, 'xrr': xrr
# #                         'std': std, 'stk': stk, 'stsm': stsm, 'stt': slStT.value
#                            }),
#             "strategy": "2EMASTH",
#             "profit": floor(100* (trades.iloc[-1].cap / capital - 1)) if len(trades.index) else 0. 
#         })
#     )); f.write("\n");
    
    
    plt.close("all")
    
    with plt.rc_context():
        kwargs = dict(warn_too_much_data=100000)
        # fig = mpf.figure(figsize=(15,6))
        # ax1 = fig.add_subplot(2,1,1)
        # ax11 = ax1.twinx()

        ap2 = [ 
                mpf.make_addplot(wdf.cum_pnl,color='g',panel=1),
                # mpf.make_addplot(tdf.capital,color='g',panel=1),
                mpf.make_addplot(wdf.enter_price,type='scatter',markersize=20,marker='^',color='black', panel=0),
                mpf.make_addplot(wdf.exit_price,type='scatter',markersize=20,marker='v', color='black',panel=0),
                mpf.make_addplot(wdf.mav2,type='line',panel=0),
                mpf.make_addplot(wdf.mav1,type='line', panel=0),
                mpf.make_addplot(wdf.mav3,type='line', panel=0),
                # mpf.make_addplot(wdf.cum_pnl,color='b',panel=1)
                # mpf.make_addplot(df['LowerB'],color='b',panel=1),
            ]
        mpf.plot(wdf,**{**kwargs, **dict(volume=False, style='binance',type='candle', figsize=(15, 6))},addplot=ap2)

        # mpf.plot(hdf,ax=ax1,**kwargs, style='binance')

        wclose_min = wdf.close.min()
        wclose_max = wdf.close.max()
       
    plt.show()
    # print(pf.stats())
    # print(pf.returns_stats())
    
if exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];
                                        

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()})
display(ui, out)

Output()

In [20]:
buy_sig

False

In [81]:
tdf.to_csv('2-emas-pnl.csv')